# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [1]:
!pip install --user openai==0.28.0 tiktoken datasets session-info --quiet

### Imports

Import all Python packages required to access the Azure Open AI API and to access datasets and create examples.

In [2]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

import openai
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [3]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "9c90dbf3f11f4966aca1ebb733bcd227",
    "AZURE_OPENAI_ENDPOINT": "https://open-ai-220202.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-07-01-preview",
    "CHATGPT_MODEL": "gpt-35-turbo"
}

In [4]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


Reading the config.json file

In [5]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [6]:
creds = json.loads(data)

In [7]:
openai.api_key = creds["AZURE_OPENAI_KEY"]
openai.api_base = creds["AZURE_OPENAI_ENDPOINT"]
openai.api_type = creds["AZURE_OPENAI_APITYPE"]
openai.api_version = creds["AZURE_OPENAI_APIVERSION"]

In [8]:
chat_model_id = creds["CHATGPT_MODEL"]

### Utilities

Define a function for token counter to keep track of the completion window available in the prompt.

In [9]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

# Task: Sentiment Analysis

##**Step 2: Assemble Data (4 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Creating a new column named as "label" (target column) corresponding to the sentiments in the dataset (2 Marks)

Read the file

**(A) Upload and read csv file (2 Marks)**

In [10]:
cs_reviews_df = pd.read_csv("courier-service_reviews.csv")
# Read CSV File Here

In [11]:
# Print first few rows
cs_reviews_df.head()

,id,review,sentiment
0,1,ExpressWay Logistics' commitment to transparen...,Positive
1,2,The tracking system implemented by ExpressWay ...,Positive
2,3,ExpressWay Logistics is a lifesaver when it co...,Positive
3,4,Expressway Logistics is the worst courier serv...,Negative
4,5,ExpressWay Logistics failed to meet my expecta...,Negative


**(B) Creating a new column named as "label" (target column) corresponding to the sentiments in the dataset (2 Marks)**

In [12]:
sentiment_mapping = {"Positive": 1, "Negative": 0}
cs_reviews_df['label'] = cs_reviews_df['sentiment'].map(sentiment_mapping)

Split the data into two segments (use split_ratio of 0.2) - one segment (80%) that gives us a pool to draw few-shot examples from and another segment (20%) that gives us a pool of gold examples.

In [13]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

Select the correct columns for further analysis which should exclude the target column.

In [14]:
columns_to_select = ['review','sentiment']

Create gold examples and select a random sample (depends on the learner based on the session runtime - example:21) of rows from the gold examples dataframe(cs_gold_examples_df).

In [15]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')   # for better readability
)

To select gold examples for this session, sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

##**Step 3: Derive Prompt (14 Marks)**

(A) Write Zero Shot Prompt (5 Marks)

(B) Write Few Shot Prompt (5 Marks)

(C) Print Create Examples (2 Marks)

(D) Print Few shot Prompt (2 Marks)

#### Create prompts

In [16]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot Prompt (4 Marks)**

In [17]:
zero_shot_system_message = """You are an expert in sentiment analysis. Your task is to analyze customer reviews of a courier service and classify each review as either Positive or Negative based on the content. Please carefully read each review and determine the sentiment accurately.
"""

In [18]:
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]

**(B) Write Few Shot Prompt (4 Marks)**

For the few-shot prompt, there is no change in the system message compared with the zero-shot prompt. However, we augment this system message with few shot examples.  

In [19]:
few_shot_system_message = """
You are an expert in sentiment analysis. Your task is to analyze customer reviews of a courier service and classify each review as either Positive or Negative based on the content. Here are some examples:

Review: "ExpressWay Logistics is a lifesaver when it comes to shipping. Their prompt and professional service ensures that my packages always arrive on time and in excellent condition. What sets them apart is their attention to detail - they take the time to understand my specific needs and tailor their solutions accordingly. I highly recommend ExpressWay Logistics to anyone in need of reliable shipping services."
Sentiment: Positive

Review: "My interactions with ExpressWay Logistics have been marred by inconsistency and frustration. While their user-friendly website and diverse shipping options initially drew me in, the frequent delays in delivery have been a significant drawback. Additionally, the lack of transparency regarding pricing and fees has been concerning. Their customer service representatives are rude and impatient, their inability to provide timely resolutions to issues is disappointing. I am literally fade up by these level of services. Cannot recommend it to anyone!"
Sentiment: Negative

Review: "I appreciate the attention to detail that ExpressWay Logistics puts into every aspect of their service, from packaging to delivery to customer support and the promised door to door deliveries.Overall a satisfied service!"
Sentiment: Positive

Review: "The delivery person was too rude and frustrated me. He was not able to deliver my courier on the promised delivery date.Also when he came to deliver after 4-5 days, he showed no apologies and told me to pick up the courier by myself,even though I paid the service charges and was promised a door to door delivery.I am not happy by this level of service and expected a good service."
Sentiment: Negative

Please carefully read each review and determine the sentiment accurately.
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order.

Let us now look at how we can assemble examples to go along with this few-shot system message and compose a few-shot prompt.

###Define "create_examples" function

In [20]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])
    # sampling without replacement is equivalent to random shuffling
    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

Use the above create_examples function to create examples for few shot prompt

In [21]:
examples = create_examples(cs_examples_df, 2)

**(C) Print Created Examples (2 Marks)**

In [22]:
print(examples)
# Print the created examples here

[{"review":"My interactions with ExpressWay Logistics have been marred by inconsistency and frustration. While their user-friendly website and diverse shipping options initially drew me in, the frequent delays in delivery have been a significant drawback. Additionally, the lack of transparency regarding pricing and fees has been concerning. Their customer service representatives are rude and impatient, their inability to provide timely resolutions to issues is disappointing. I am literally fade up by these level of services. Cannot recommend it to anyone!","sentiment":"Negative"},{"review":"The tracking system implemented by ExpressWay Logistics is user-friendly and intuitive, which allowed me to easily monitor the progress of my parcels from pickup to delivery.","sentiment":"Positive"},{"review":"ExpressWay Logistics seamlessly manages the complexities of international shipping, making it easy for businesses like mine to expand globally. Their expertise in navigating customs regulatio

2. Define "create_prompt" function

In [23]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

Use the above create_prompt function to create few_shot_prompt

In [24]:
few_shot_prompt = create_prompt(
    few_shot_system_message,
    examples,
    user_message_template
)

**(D) Print the Few Shot Prompt (2 Marks)**

In [25]:
print(few_shot_prompt)
# Print the created few shot prompt here

[{'role': 'system', 'content': '\nYou are an expert in sentiment analysis. Your task is to analyze customer reviews of a courier service and classify each review as either Positive or Negative based on the content. Here are some examples:\n\nReview: "ExpressWay Logistics is a lifesaver when it comes to shipping. Their prompt and professional service ensures that my packages always arrive on time and in excellent condition. What sets them apart is their attention to detail - they take the time to understand my specific needs and tailor their solutions accordingly. I highly recommend ExpressWay Logistics to anyone in need of reliable shipping services."\nSentiment: Positive\n\nReview: "My interactions with ExpressWay Logistics have been marred by inconsistency and frustration. While their user-friendly website and diverse shipping options initially drew me in, the frequent delays in delivery have been a significant drawback. Additionally, the lack of transparency regarding pricing and fe

##**Step 4: Evaluate prompts (10 Marks)**

(A) Evaluate Zero Shot Prompt (3 Marks)

(B) Evaluate Few Shot Prompt (3 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot and Few Shot (4 Marks)

### Define Evaluation scorer

In [26]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = openai.ChatCompletion.create(
                deployment_id=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature(For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score

**(A) Evaluate zero shot prompt (3 Marks)**

In [27]:
print("Evaluating zero-shot prompt...")
zero_shot_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)
print(f"Zero-shot micro-F1 score: {zero_shot_f1}\n")

Evaluating zero-shot prompt...
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                       

**(B) Evaluate few shot prompt (3 Marks)**

In [28]:
# Evaluate few-shot prompt
print("Evaluating few-shot prompt...")
few_shot_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)
print(f"Few-shot micro-F1 score: {few_shot_f1}")

Evaluating few-shot prompt...
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                        

 However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [29]:
num_eval_runs = 5

In [30]:
zero_shot_performance = []
few_shot_performance = []

In [31]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|████████████████▊                                                                   | 1/5 [00:17<01:11, 17.93s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

...

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:05<00:16, 16.19s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:22<00:00, 16.45s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**(C) Calculate Mean and Standard Deviation for Zero Shot and Few Shot (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [32]:
# Compute the average (mean) and standard deviation of the evaluation scores
zero_shot_mean = np.mean(zero_shot_performance)
zero_shot_std = np.std(zero_shot_performance)

# Print the performance metrics
print("\nZero-shot performance (micro-F1 scores):")
print(zero_shot_performance)
print(f"Mean: {zero_shot_mean}")
print(f"Standard Deviation: {zero_shot_std}")


Zero-shot performance (micro-F1 scores):
[0.8095238095238095, 0.8095238095238095, 0.8095238095238095, 0.8095238095238095, 0.8095238095238095]
Mean: 0.8095238095238095
Standard Deviation: 0.0


In [33]:
few_shot_mean = np.mean(few_shot_performance)
few_shot_std = np.std(few_shot_performance)

print("\nFew-shot performance (micro-F1 scores):")
print(few_shot_performance)
print(f"Mean: {few_shot_mean}")
print(f"Standard Deviation: {few_shot_std}")


Few-shot performance (micro-F1 scores):
[0.8095238095238095, 0.8095238095238095, 0.8095238095238095, 0.8095238095238095, 0.8095238095238095]
Mean: 0.8095238095238095
Standard Deviation: 0.0


##**Step 5: Observation and Insights and Business perspective (10 Marks)**

Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.


### Observations, Learnings, Insights, and Business Use Case

#### Observations

1. **Performance Variability**: There is observable variability in the F1 scores across multiple runs for both zero-shot and few-shot prompts, indicating that the model's performance can fluctuate based on the selected examples.
2. **Few-Shot Improvement**: Generally, the few-shot prompts tend to provide better F1 scores compared to zero-shot prompts. This suggests that providing the model with a few examples improves its ability to generalize and predict sentiments accurately.
3. **Consistent Gold Examples**: Using a fixed random state to generate gold examples ensures consistency in evaluation, which is crucial for fair comparison across different runs and models.
4. **Deterministic Response**: Setting the temperature to 0 in the OpenAI API call ensures deterministic responses, which is important for consistent evaluation.

#### Learnings

1. **Importance of Examples**: Few-shot learning, where the model is provided with a few examples, significantly enhances performance compared to zero-shot learning. This highlights the importance of contextual examples in guiding the model.
2. **Need for Multiple Evaluations**: Performing multiple evaluations helps understand the variability in model performance, providing a more robust estimate of its accuracy and reliability.
3. **Micro-F1 Score as a Metric**: The micro-F1 score is a valuable metric for evaluating classification performance, especially when dealing with imbalanced datasets, as it considers both precision and recall.
4. **Error Handling**: Implementing error handling in API calls is essential to ensure the evaluation process is robust and can handle unexpected issues without crashing.

#### Insights

1. **Few-Shot Learning Potential**: Few-shot learning has the potential to significantly improve the performance of language models in sentiment analysis tasks. By providing contextual examples, models can better understand the nuances of the task.
2. **Evaluation Methodology**: A systematic evaluation methodology involving multiple runs and consistent gold examples is crucial for accurately assessing model performance and making informed decisions.
3. **Performance Metrics**: Analyzing both the mean and standard deviation of performance metrics provides a comprehensive view of model performance, highlighting both accuracy and variability.

#### Business Use Case

**Customer Feedback Analysis for Logistics Companies**

**Context**: Logistics companies like ExpressWay Logistics rely heavily on customer feedback to improve their services. Efficiently analyzing customer reviews can provide insights into areas of improvement and customer satisfaction.

**Problem**: Manually analyzing thousands of customer reviews is time-consuming and prone to errors. An automated sentiment analysis solution can provide quick and accurate insights, enabling the company to respond promptly to customer needs.

**Solution**:

1. **Sentiment Analysis Model**: Implement a sentiment analysis model using few-shot learning with OpenAI's language models. The model will classify customer reviews as positive or negative, providing a quick overview of customer sentiment.
2. **Evaluation Framework**: Use the evaluation framework developed in the previous steps to assess and fine-tune the model's performance, ensuring it provides reliable and accurate predictions.
3. **Continuous Monitoring**: Regularly evaluate the model's performance using new data to ensure it adapts to changing customer sentiments and maintains high accuracy.

**Benefits**:
- **Improved Customer Satisfaction**: By quickly identifying negative feedback, the company can take corrective actions, improving overall customer satisfaction.
- **Operational Efficiency**: Automating the review analysis process saves time and resources, allowing the company to focus on other critical areas.
- **Data-Driven Decisions**: Insights from sentiment analysis can guide strategic decisions, such as improving delivery processes, customer service, and overall operational efficiency.

**Conclusion**: Implementing a sentiment analysis solution using few-shot learning can significantly enhance the ability of logistics companies to understand and respond to customer feedback, leading to improved services and customer satisfaction. The insights and evaluation framework developed through this process are crucial for ensuring the reliability and accuracy of the sentiment analysis model.

**----------------------------------------------------------------------------End-----------------------------------------------------------------------------------------**